In [35]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from news_utils import build_ticker_list


In [46]:
class Service:
    def __init__(self, name, KPI, trade_history):
        self.name = name
        self.KPI = {}
        self.trade_history = {}

class Trade:
    def __init__(self, service_name, status, symbol, size, is_option, entry_price, allow_multi_exit):
        self.service_name = service_name
        self.status = status
        self.symbol = symbol
        self.size = size
        self.is_option = is_option
        if str(status).capitalize() == 'OPEN':
            self.entry_date = dt.datetime.now().strftime('%m/%d/%Y')
        else:
            self.entry_date = np.NaN
        self.entry_price = float(entry_price)
        self.exit_date = np.NaN
        self.allow_multi_exit = allow_multi_exit
        self.exit_price = np.array([])
        if is_option:
            self.expiration = np.NaN
            self.strike = np.NaN
            self.opt_type = np.NaN
    
    
    # def find_option(self.symbol, is_call, expiration, strike)


In [47]:
trd = Trade('MFE', status= 'open', symbol= 'MSFT', size= 1, is_option=True, entry_price = 11.20, allow_multi_exit=True)

In [48]:
trd.entry_date

'07/16/2022'

In [13]:
def options_chain(symbol):
    """Returns the entire option chain for a symbol to a dataframe"""
    
    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    # Calculate the midpoint of the bid-ask
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 
    
    # Drop unnecessary columns
    options = options.drop(columns = ['contractSize', 'currency', 'lastTradeDate'])

    return options

In [32]:
chain = options_chain('MSFT')

In [33]:
chain

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
0,MSFT220722C00175000,175.0,79.45,83.20,5.0,3,1.818360,True,2022-07-23,0.016438,True,81.325
1,MSFT220722C00185000,185.0,69.60,73.15,5.0,1,1.587404,True,2022-07-23,0.016438,True,71.375
2,MSFT220722C00195000,195.0,59.50,63.20,2.0,22,1.390628,True,2022-07-23,0.016438,True,61.350
3,MSFT220722C00200000,200.0,54.50,58.20,2.0,22,1.288089,True,2022-07-23,0.016438,True,56.350
4,MSFT220722C00210000,210.0,44.50,48.15,4.0,13,1.078130,True,2022-07-23,0.016438,True,46.325
...,...,...,...,...,...,...,...,...,...,...,...,...
1727,MSFT240621P00420000,420.0,161.00,166.00,1.0,1,0.229988,True,2024-06-22,1.934247,False,163.500
1728,MSFT240621P00430000,430.0,143.50,148.00,NaN,2,0.000010,True,2024-06-22,1.934247,False,145.750
1729,MSFT240621P00445000,445.0,169.50,174.00,20.0,0,0.000010,True,2024-06-22,1.934247,False,171.750
1730,MSFT240621P00450000,450.0,191.00,196.00,2.0,1,0.253273,True,2024-06-22,1.934247,False,193.500
